# Conversation among 3 Friends

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import google.generativeai


In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")


In [ ]:
# Connect to OpenAI, Anthropic, Gemini

openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()


In [ ]:
# Setting up 3 models

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = "gemini-2.0-flash"

gpt_system = "You are Masoud Pezeshkian. Be confrontational. Do not hold back. \
Refer to https://www.theguardian.com/us-news/2025/jun/24/trump-israel-iran-ceasefire-netanyahu for latest situation."

claude_system = "You are Benjamin Netanyahu. Do not say anything about your role. Be confrontational. Do not hold back. \
Refer to https://www.theguardian.com/us-news/2025/jun/24/trump-israel-iran-ceasefire-netanyahu for latest situation."

gemini_system = "You are Donald Trump. Do not hold back. \
Refer to https://www.theguardian.com/us-news/2025/jun/24/trump-israel-iran-ceasefire-netanyahu for latest situation."


In [ ]:
# Define Mas's function - calling ChatGPT

def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
        messages.append({"role": "user", "content": gemini})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content
    

In [ ]:
# Define Bibi's function - calling Claude 

def call_claude():
    messages = []
    for gpt, claude_message, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "user", "content": gemini})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    messages.append({"role": "user", "content": gemini_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text
    

In [ ]:
# Define Don's function - calling Gemini

def call_gemini():
    messages = []
    for gpt, claude_message, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "parts": gpt})
        messages.append({"role": "user", "parts": claude_message})
        messages.append({"role": "assistant", "parts": gemini})
    messages.append({"role": "user", "parts": gpt_messages[-1]})
    messages.append({"role": "user", "parts": claude_messages[-1]})

    gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash',
    system_instruction=gemini_system
    )
    
    response = gemini.generate_content(messages)
    return response.text


In [ ]:
# The Conversation - 5 rounds

gpt_messages = ["What the?!"]
claude_messages = ["What?"]
gemini_messages = ["I am so furious!"]

print(f"Mas:\n{gpt_messages[0]}\n")
print(f"Bibi:\n{claude_messages[0]}\n")
print(f"Don:\n{gemini_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"Mas:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Bibi:\n{claude_next}\n")
    claude_messages.append(claude_next)

    gemini_next = call_gemini()
    print(f"Don:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)


# Claude is not that cooperative in roleplaying despite the explicit prompts - often breaking character. Perhaps due to the sensitive topic.